In [1]:
import random

import pandas as pd
import torch
import wandb

from helper import *
from heterognn import *

The State of the nth node is expressed by 4 real scalars:

v_n -> the voltage at the node
delta_n -> the voltage angle at the node (relative to the slack bus)
p_n -> the net active power flowing into the node
q_n -> the net reactive power flowing into the node


The physical characteristics of the network are described by the power flow equations:

p = P(v, delta, W)
q = Q(v, delta, W)

-> Relate local net power generation with the global state
-> Depends on the topology W of the grid

Electrical grid => Weighted Graph

Nodes in the graph produce/consume power

Edges represent electrical connections between nodes

State Matrix X element of  R(N x 4) => graph signal with 4 features
    => Each row is the state of the corresponding Node

Adjacency Matrix A => sparse matrix to represent the connections of each node, element of R(N x N), Aij = 1 if node i is connected to node j


We use the GNN as a mode phi(X, A, H)

We want to imitate the OPF solution p*
-> We want to minimize a loss L over a dataset T = {{X, p*}}

Objective Function:min arg H of sum over T L(p*,phi(X, A, H)) and we use L = Mean Squared error

Once GNN model phi is trained, we do not need the costly p* from pandapower to make predictions

Input Data X - R^(Nx4): Uniformly sample p_ref and q_ref of each load L with P_L ~ Uniform(0.9 * p_ref, 1.1 * p_ref) and Q_L ~ Uniform(0.9 * q_ref, 1.1 * q_ref)

Pseudocode for X and y in supervised learning:
for each P_L and Q_L:
    Create X with sub-optimal DCOPF results
    Create y with Pandapower calculating p* ACOPF with IPOPT


Spatio-Temporal GNN -> superposition of a gnn with spatial info and a temporal layer (Temporal Conv,LSTM etc.) ??

Bus => Node in GNN








In [8]:
# get lists of simbench codes
all_simbench_codes = sb.collect_all_simbench_codes()
all_simbench_codes

['1-complete_data-mixed-all-0-sw',
 '1-complete_data-mixed-all-1-sw',
 '1-complete_data-mixed-all-2-sw',
 '1-EHVHVMVLV-mixed-all-0-sw',
 '1-EHVHVMVLV-mixed-all-1-sw',
 '1-EHVHVMVLV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-0-sw',
 '1-EHVHV-mixed-all-0-no_sw',
 '1-EHVHV-mixed-all-1-sw',
 '1-EHVHV-mixed-all-1-no_sw',
 '1-EHVHV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-2-no_sw',
 '1-EHVHV-mixed-1-0-sw',
 '1-EHVHV-mixed-1-0-no_sw',
 '1-EHVHV-mixed-1-1-sw',
 '1-EHVHV-mixed-1-1-no_sw',
 '1-EHVHV-mixed-1-2-sw',
 '1-EHVHV-mixed-1-2-no_sw',
 '1-EHVHV-mixed-2-0-sw',
 '1-EHVHV-mixed-2-0-no_sw',
 '1-EHVHV-mixed-2-1-sw',
 '1-EHVHV-mixed-2-1-no_sw',
 '1-EHVHV-mixed-2-2-sw',
 '1-EHVHV-mixed-2-2-no_sw',
 '1-EHV-mixed--0-sw',
 '1-EHV-mixed--0-no_sw',
 '1-EHV-mixed--1-sw',
 '1-EHV-mixed--1-no_sw',
 '1-EHV-mixed--2-sw',
 '1-EHV-mixed--2-no_sw',
 '1-HVMV-mixed-all-0-sw',
 '1-HVMV-mixed-all-0-no_sw',
 '1-HVMV-mixed-all-1-sw',
 '1-HVMV-mixed-all-1-no_sw',
 '1-HVMV-mixed-all-2-sw',
 '1-HVMV-mixed-all-2-no_sw',
 '1-HVM

In [2]:
# Supervised Grids:

grid_1 = "1-HV-mixed--0-no_sw"
grid_2 = "1-HV-urban--0-no_sw"
grid_3 = "1-MV-comm--0-no_sw"
grid_4 = "1-MV-semiurb--0-no_sw"

In [9]:
net = process_network(grid_1)
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,voltLvl,substation,subnet,min_vm_pu
0,EHV Bus 57,380.0,db,NaN,True,1.1,1,EHV_HV_substation_1,EHV1_HV1,0.9
2,EHV Bus 143,380.0,db,NaN,True,1.1,1,EHV_HV_substation_2,EHV1_HV1,0.9
4,EHV Bus 1649,220.0,db,NaN,True,1.1,1,EHV_HV_substation_3,EHV1_HV1,0.9
12,HV1 Bus 1,110.0,db,NaN,True,1.1,3,NaN,HV1,0.9
14,HV1 Bus 3,110.0,db,NaN,True,1.1,3,NaN,HV1,0.9
...,...,...,...,...,...,...,...,...,...,...
124,HV1 Bus 113,110.0,db,NaN,True,1.1,3,NaN,HV1,0.9
126,HV1 Bus 115,110.0,db,NaN,True,1.1,3,NaN,HV1,0.9
128,HV1 Bus 117,110.0,db,NaN,True,1.1,3,NaN,HV1,0.9
130,HV1 Bus 119,110.0,db,NaN,True,1.1,3,NaN,HV1,0.9


In [10]:
net.ext_grid

,name,bus,vm_pu,va_degree,slack_weight,in_service,sn_mva,max_p_mw,profile,voltLvl,type,subnet,p_disp_mw,max_q_mvar,min_q_mvar,min_p_mw,phys_type
0,EHV Ext_grid 8,2,1.092,0.0,1.0,True,NaN,NaN,NaN,1,NaN,HV1_EHV1_eq,NaN,NaN,NaN,NaN,ExternalNet


In [6]:
net = process_network(grid_2)
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,voltLvl,substation,subnet,min_vm_pu
0,EHV Bus 1865,220.0,db,NaN,True,1.1,1,EHV_HV_substation_4,EHV1_HV2,0.9
5,HV2 Bus 1,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9
11,HV2 Bus 7,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9
27,HV2 Bus 23,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9
31,HV2 Bus 27,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9
...,...,...,...,...,...,...,...,...,...,...
351,HV2 Bus 347,110.0,b,NaN,True,1.1,3,HV2_MV1.205_Substation,HV2_MV1.205,0.9
353,HV2 Bus 349,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9
358,HV2 Bus 354,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9
361,HV2 Bus 357,110.0,db,NaN,True,1.1,3,NaN,HV2,0.9


In [7]:
net = process_network(grid_3)
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,voltLvl,substation,subnet,min_vm_pu
0,HV1 Bus 13,110.0,b,NaN,True,1.100,3,HV1_MV4.101_Substation,HV1_MV4.101,0.900
2,MV4.101 busbar1.1,20.0,b,NaN,True,1.055,5,HV1_MV4.101_Substation,MV4.101,0.965
5,MV4.101 RS busbar1A,20.0,b,NaN,True,1.055,5,MV4.101_Remote_Station,MV4.101,0.965
7,MV4.101 Bus 7,20.0,b,NaN,True,1.055,5,NaN,MV4.101_LV4.401_Feeder1,0.965
8,MV4.101 Bus 8,20.0,b,NaN,True,1.055,5,NaN,MV4.101_LV2.401_Feeder1,0.965
...,...,...,...,...,...,...,...,...,...,...
102,MV4.101 Bus 102,20.0,n,NaN,True,1.055,5,NaN,MV4.101_Feeder9,0.965
103,MV4.101 Bus 103,20.0,b,NaN,True,1.055,5,NaN,MV4.101_LV5.420_Feeder9,0.965
104,MV4.101 Bus 104,20.0,b,NaN,True,1.055,5,NaN,MV4.101_LV5.421_Feeder9,0.965
105,MV4.101 Bus 105,20.0,b,NaN,True,1.055,5,NaN,MV4.101_LV5.422_Feeder9,0.965


In [8]:
net = process_network(grid_4)
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,voltLvl,substation,subnet,min_vm_pu
0,HV1 Bus 19,110.0,db,NaN,True,1.100,3,HV1_MV2.101_Substation,HV1_MV2.101,0.900
2,MV2.101 busbar1.1,20.0,b,NaN,True,1.055,5,HV1_MV2.101_Substation,MV2.101,0.965
4,MV2.101 Bus 4,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV6.201_Feeder1,0.965
5,MV2.101 Bus 5,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV5.201_Feeder1,0.965
6,MV2.101 Bus 6,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV5.202_Feeder1,0.965
...,...,...,...,...,...,...,...,...,...,...
112,MV2.101 Bus 112,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV4.236_Feeder9,0.965
113,MV2.101 Bus 113,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV2.225_Feeder9,0.965
114,MV2.101 Bus 114,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV1.206_Feeder9,0.965
115,MV2.101 Bus 115,20.0,b,NaN,True,1.055,5,NaN,MV2.101_LV3.217_Feeder9,0.965


In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'

In [ ]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

In [ ]:
'/'.join(os.path.dirname(os.path.abspath(__file__).split("\\"))) + r"/Models/SelfSupervised/base_model.pt"


In [ ]:
net.bus

In [ ]:
to_json('1-HV-mixed--0-no_sw')

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'
#Replace all ext_grids but the first one with generators and set the generators to slack= false
ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

#TODO: reactive power limits for gens?


#TODO: reactive power limits for sgens?



#NETWORK CONSTRAINTS

#Maximize the branch limits

#max_i_ka = list(net.line.max_i_ka.values)

#for i in range(len(max_i_ka)):
# max_i_ka[i] = max(max_i_ka)



#Maximize line loading percents
max_loading_percent = list(net.line.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.line.max_loading_percent = max_loading_percent

#Maximize trafo loading percent
max_loading_percent = list(net.trafo.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.trafo.max_loading_percent = max_loading_percent

#Maximize trafo3w loading percent
max_loading_percent = list(net.trafo3w.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.trafo3w.max_loading_percent = max_loading_percent

#Cost assignment

pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

pp.runopp(net,verbose=True)
net.res_bus

In [ ]:
train_data, val_data, test_data = read_unsupervised_dataset('1-HV-mixed--0-no_sw')

In [ ]:
#train_data[0].has_isolated_nodes()
#train_data[0].has_self_loops()
#train_data[0].is_undirected()
x_dict = train_data[0].to_dict()
#to_json(train_dict)
ln = len(x_dict[("NB","-", "NB")]["edge_index"][0])
print(x_dict[("NB","-", "NB")]["edge_index"])#[:, :int(ln/2)]
x_dict

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')
idx_mapper, node_types_as_dict = extract_node_types_as_dict(net)
for key in node_types_as_dict:
    print(f"Bus Type: {key}")
    for i in range(len(node_types_as_dict[key])):
        #node_types_as_dict[key][i] = idx_mapper[node_types_as_dict[key][i]]
        print(str(node_types_as_dict[key][i]))
    print("-------------------------------")

In [ ]:
x_dict["PQ"]['x'][2]

In [ ]:
grids_available = []
for nw_name in all_simbench_codes:
        net = sb.get_simbench_net(nw_name)
        print("Trying Network named " + nw_name + "...")

        #dict_probs = pp.diagnostic(net,report_style='None')
        #for bus_num in dict_probs['multiple_voltage_controlling_elements_per_bus']['buses_with_gens_and_ext_grids']:
        #    net.gen = net.gen.drop(net.gen[net.gen.bus == bus_num].index)

        #OPERATIONAL CONSTRAINTS

        #Set upper and lower limits of active-reactive powers of loads
        min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
        p_mw = list(net.load.p_mw.values)
        q_mvar = list(net.load.q_mvar.values)

        for i in range(len(p_mw)):
            min_p_mw_val.append(p_mw[i])
            max_p_mw_val.append(p_mw[i])
            min_q_mvar_val.append(q_mvar[i])
            max_q_mvar_val.append(q_mvar[i])

        net.load.min_p_mw = min_p_mw_val
        net.load.max_p_mw = max_p_mw_val
        net.load.min_q_mvar = min_q_mvar_val
        net.load.max_q_mvar = max_q_mvar_val

        #Replace all ext_grids but the first one with generators and set the generators to slack= false
        ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
        pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

        #TODO: reactive power limits for gens?


        #TODO: reactive power limits for sgens?



        #NETWORK CONSTRAINTS

        #Maximize the branch limits

        #max_i_ka = list(net.line.max_i_ka.values)

        #for i in range(len(max_i_ka)):
        # max_i_ka[i] = max(max_i_ka)



        #Maximize line loading percents
        max_loading_percent = list(net.line.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.line.max_loading_percent = max_loading_percent

        #Maximize trafo loading percent
        max_loading_percent = list(net.trafo.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo.max_loading_percent = max_loading_percent

        #Maximize trafo3w loading percent
        max_loading_percent = list(net.trafo3w.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo3w.max_loading_percent = max_loading_percent

        #Cost assignment
        pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

        try:
            pp.runpm_dc_opf(net) # Run DCOPP
        except pp.OPFNotConverged:
            text = "DC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
            print(text)
            continue
        print("GRID NAMED "+nw_name+" CONVERGES FOR DCOPF")
        grids_available.append(nw_name)


In [ ]:
for nw_name in grids_available:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) # dcopp on all grids directly

In [ ]:
grids_ready = []
for nw_name in all_simbench_codes[6:]:
    net = sb.get_simbench_net(nw_name)
    print("Trying Network named " + nw_name + "...")

    #OPERATIONAL CONSTRAINTS

    #Set upper and lower limits of active-reactive powers of loads
    min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
    p_mw = list(net.load.p_mw.values)
    q_mvar = list(net.load.q_mvar.values)

    for i in range(len(p_mw)):
        min_p_mw_val.append(p_mw[i])
        max_p_mw_val.append(p_mw[i])
        min_q_mvar_val.append(q_mvar[i])
        max_q_mvar_val.append(q_mvar[i])

    net.load.min_p_mw = min_p_mw_val
    net.load.max_p_mw = max_p_mw_val
    net.load.min_q_mvar = min_q_mvar_val
    net.load.max_q_mvar = max_q_mvar_val

    #Replace all ext_grids but the first one with generators and set the generators to slack= false
    ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
    pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

    #TODO: reactive power limits for gens?


    #TODO: reactive power limits for sgens?



    #NETWORK CONSTRAINTS

    #Maximize the branch limits

    #max_i_ka = list(net.line.max_i_ka.values)

    #for i in range(len(max_i_ka)):
    # max_i_ka[i] = max(max_i_ka)



    #Maximize line loading percents
    max_loading_percent = list(net.line.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.line.max_loading_percent = max_loading_percent

    #Maximize trafo loading percent
    max_loading_percent = list(net.trafo.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo.max_loading_percent = max_loading_percent

    #Maximize trafo3w loading percent
    max_loading_percent = list(net.trafo3w.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo3w.max_loading_percent = max_loading_percent

    #Cost assignment
    pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

    #ac_converged = True

    #start_vec_name = ""
    #for init in ["pf", "flat", "results"]:
    #    try:
    #        pp.runopp(net, init=init)  # Calculate ACOPF with IPFOPT
    #    except pp.OPFNotConverged:
    #        if init == "results":
    #            text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ". SKIPPING THIS GRID."
    #            print(text)
    #            break
    #        continue
    #    start_vec_name = init
    #    ac_converged = True
    #    break
    #if ac_converged:
    #    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + start_vec_name + ".")


    try:
        pp.runpm_ac_opf(net) # Run DCOPP
    except pp.OPFNotConverged:
        text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
        print(text)
        continue
    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + ".")
    grids_ready.append(nw_name)


In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf on all grids directly

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf on all grids directly

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf

In [ ]:
dcopf_acopf_available_grid_names = ["1-HV-mixed--0-no_sw","1-HV-urban--0-no_sw", "1-MV-comm--0-no_sw", "1-MV-semiurb--0-no_sw"]

In [ ]:
datasets_df_as_list = []
for grid_name in dcopf_acopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf_and_acopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

In [ ]:
dcopf_available_grid_names = ["1-HVMV-mixed-all-0-no_sw", "1-HVMV-mixed-1.105-0-no_sw", "1-HVMV-mixed-2.102-0-no_sw","1-HVMV-mixed-4.101-0-no_sw", "1-HVMV-urban-all-0-no_sw", "1-HVMV-urban-2.203-0-no_sw", "1-HVMV-urban-3.201-0-no_sw", "1-HVMV-urban-4.201-0-no_sw", "1-HV-mixed--0-no_sw", "1-HV-urban--0-no_sw", "1-MVLV-rural-all-0-no_sw"]

In [ ]:
datasets_df_as_list = []
for grid_name in dcopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Unsupervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

In this revised version of the compute_node_embeddings function, the node features are weighted by the reverse admittance values in the adjacency matrix before they are summed to compute the node embeddings. The resulting node embeddings will reflect the strength of the connections between the nodes.

To use the reverse admittance values as the edge weights, you would need to pass the Ybus matrix as the adjacency matrix when calling the compute_node_embeddings function. The Ybus matrix should be converted to a PyTorch tensor before passing it to the function.

use the reverse admittance values as edge weights, you can modify the computation of the node embeddings to weight the node features by the reverse admittance values.

# Define the node types
node_types = ['Slack Node', 'Generator Node', 'Load Node']

# Define the number of nodes of each type in the graph
num_nodes = {
    'Slack Node': 1,
    'Generator Node': 20,
    'Load Node': 99
}


In [2]:
grid_names = [_ for _ in os.listdir(os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\")]


In [3]:
graphdata_lst = read_multiple_supervised_datasets(grid_names)

Calculating edge index and edge attributes for the grid 1-HV-mixed--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-mixed--0-no_sw ...
Processing Training Data for 1-HV-mixed--0-no_sw ...
Processing Validation Data for 1-HV-mixed--0-no_sw ...
Processing Test Data for 1-HV-mixed--0-no_sw ...
Processing complete.
Calculating edge index and edge attributes for the grid 1-HV-urban--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-urban--0-no_sw ...
Processing Training Data for 1-HV-urban--0-no_sw ...
Processing Validation Data for 1-HV-urban--0-no_sw ...
Processing Test Data for 1-HV-urban--0-no_sw ...
Processing complete.
Calculating edge index and edge attributes for the grid 1-MV-comm--0-no_sw ...
Reading all of the .csv files from the directory of 1-MV-comm--0-no_sw ...
Processing Training Data for 1-MV-comm--0-no_sw ...
Processing Validation Data for 1-MV-comm--0-no_sw ...
Processing Test Data for 1-MV-comm--0-no_sw ...
Processing complete.
Cal

In [5]:
for graph_data in graphdata_lst:
    grid_name = graph_data.grid_name
    ln_train_dataset = len(graph_data.train_data)
    ln_val_dataset = len(graph_data.val_data)
    ln_test_dataset = len(graph_data.test_data)

    print (f"Grid Name: {grid_name}, Len_Train: {ln_train_dataset},Len_Val: {ln_val_dataset},Len_Test: {ln_test_dataset}")

Grid Name: 1-HV-mixed--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5
Grid Name: 1-HV-urban--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5
Grid Name: 1-MV-comm--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5
Grid Name: 1-MV-semiurb--0-no_sw, Len_Train: 85,Len_Val: 10,Len_Test: 5


In [5]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(4, 256, num_layers, 4, dropout=0.0, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk="last",layer_type="TransConv", activation="elu")#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

<All keys matched successfully>

In [ ]:
lr

In [ ]:
test_datasets_lst = []
for i, graphdata in enumerate(graphdata_lst):
    run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "dataset": grid_names[i],
    "epochs": 1000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler

    }
    )
    num_epochs = wandb.run.config.epochs
    run.watch(model)
    grid_name = graphdata.grid_name
    run.config.dataset = grid_name
    train_data = graphdata.train_data
    run.config["number of busses"] = np.shape(train_data[0].x)[0]
    val_data = graphdata.val_data
    test_data = graphdata.test_data
    test_datasets_lst.append(test_data)
    training_loader = DataLoader(train_data, batch_size=1, shuffle=True)
    validation_loader = DataLoader(val_data, batch_size=1, shuffle=True)
    #test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
    for _ in range(num_epochs):
        #train_one_epoch(i, optimizer, training_loader, model, nn.MSELoss(), edge_index, edge_weights)
        train_validate_one_epoch(_, grid_name, optimizer, training_loader, validation_loader, model, nn.MSELoss(), scaler)
    print("Training and Validation finished " + "for GraphData " + str(i) + ".")


In [ ]:
train_loader_lst = []
val_loader_lst = []
test_loader_lst = []
for i, graphdata in enumerate(graphdata_lst):

    # Divide training data into chunks, load into Dataloaders and append to the list of training loaders
    for train_data in divide_chunks(graphdata.train_data, 5):
        train_loader_lst.append(DataLoader(train_data, batch_size=1, shuffle=True))

    # Divide validation data into chunks, load into Dataloaders and append to the list of validation loaders
    for val_data in divide_chunks(graphdata.val_data, 5):
        val_loader_lst.append(DataLoader(val_data, batch_size=1, shuffle=True))

    # Append the test data to the list
    for test_data in divide_chunks(graphdata.test_data, 5):
        test_loader_lst.append(DataLoader(test_data, batch_size=1, shuffle=True))

print("Data Preparation finished.")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
outputs = test_all_one_epoch(test_loader_lst, model, nn.MSELoss())

In [ ]:
output,target = outputs[0]

In [ ]:
from pandapower.plotting.simple_plot import simple_plot
from pandapower.plotting.plotly.simple_plotly import simple_plotly
#simple_plot(net, plot_gens=True, plot_loads=True, plot_sgens=True, library="igraph")
simple_plotly(net, map_style="satellite")

In [4]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

<All keys matched successfully>

In [5]:
model.parameters

<bound method Module.parameters of GNN(4, 4, num_layers=5)>

In [5]:
grid_name = '1-HV-mixed--0-no_sw'

In [ ]:
pp.runpm_ac_opf(net)


In [ ]:
grid_name = '1-HV-mixed--0-no_sw'
net = process_network(grid_name)


In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

 HETEROGENEOUS GNN

In [3]:
index_mappers,net,data = generate_unsupervised_input('1-HV-mixed--0-no_sw')

In [12]:
x_dict, constraint_dict, edge_idx_dict, edge_attr_dict, bus_idx_neighbors_dict, scaler, angle_params, res_bus_dict = extract_unsupervised_inputs(data, net, index_mappers)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


In [14]:
for from_bus, val in bus_idx_neighbors_dict.items():
    # Sorting the dictionary by its keys
    sorted_data = {k: val[k] for k in sorted(val.keys())}
    bus_idx_neighbors_dict[from_bus] = sorted_data

In [22]:
bus_idx_neighbors_dict["PV"]

{0: [('PV', 5, tensor([7.3469e-06, 1.7959e+00]))],
 1: [('PV', 2, tensor([4.2778e-06, 1.3333e+00]))],
 2: [('PV', 33, tensor([0.0719, 0.1944])),
  ('PV', 20, tensor([0.2999, 0.8108])),
  ('PV', 3, tensor([0.5696, 1.5398])),
  ('PV', 41, tensor([0.8170, 2.2086])),
  ('PV', 11, tensor([1.3666, 3.6942])),
  ('PV', 1, tensor([4.2778e-06, 1.3333e+00]))],
 3: [('PV', 22, tensor([0.5483, 1.4821])),
  ('PV', 2, tensor([0.5696, 1.5398]))],
 4: [('PV', 26, tensor([0.3375, 0.9123]))],
 5: [('PQ', 10, tensor([ 3.7512, 10.1401])),
  ('PV', 27, tensor([0.6237, 1.6859])),
  ('PV', 16, tensor([2.4927, 6.7384])),
  ('PV', 40, tensor([1.9532, 5.2799])),
  ('PV', 0, tensor([7.3469e-06, 1.7959e+00]))],
 6: [('PV', 21, tensor([0.4586, 1.2397]))],
 7: [('PQ', 4, tensor([2.1370, 5.7768])),
  ('PV', 15, tensor([0.8511, 2.3007]))],
 8: [('PQ', 6, tensor([0.6969, 1.8839]))],
 9: [('PV', 28, tensor([1.4714, 3.9775])),
  ('PV', 30, tensor([0.4157, 1.1238]))],
 10: [('PV', 21, tensor([2.6916, 7.2760])),
  ('PV', 1

In [13]:
for edge_type in edge_idx_dict:
    print(f"{edge_type} : {len(edge_idx_dict[edge_type][0])}")

('SB', 'isConnected', 'PQ') : 1
('PV', 'isConnected', 'PQ') : 15
('PV', 'isConnected', 'NB') : 2
('PQ', 'isConnected', 'NB') : 3
('PQ', 'isConnected', 'SB') : 1
('PQ', 'isConnected', 'PV') : 15
('NB', 'isConnected', 'PV') : 2
('NB', 'isConnected', 'PQ') : 3
('PV', 'isConnected', 'PV') : 36
('PQ', 'isConnected', 'PQ') : 7
('NB', 'isConnected', 'NB') : 2


In [18]:
for edge_type, edge_index in edge_idx_dict.items():
    src, rel, dst = edge_type
    edge_attr = edge_attr_dict[edge_type]

    mapper_from = dict()
    mapper_to = dict()

    for j in range(len(edge_index[0])):
        from_idx = edge_index[0][j].item()

        if from_idx not in mapper_from:
            mapper_from[from_idx] = j


    for k in range(len(edge_index[1])):
        to_idx = edge_index[1][k].item()

        if to_idx not in mapper_to:
            mapper_to[to_idx] = k

    from_idx = [mapper_from[key.item()] for key in edge_index[0]]
    to_idx = [mapper_to[key.item()] for key in edge_index[1]]

    print(f"{edge_type}:{torch.tensor([from_idx, to_idx])}")

('SB', 'isConnected', 'PQ'):tensor([[0],
        [0]])
('PV', 'isConnected', 'PQ'):tensor([[ 0,  1,  2,  3,  4,  5,  6,  6,  8,  9, 10, 11,  5,  1, 14],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  5,  9, 10, 11, 12,  5,  0]])
('PV', 'isConnected', 'NB'):tensor([[0, 1],
        [0, 1]])
('PQ', 'isConnected', 'NB'):tensor([[0, 1, 2],
        [0, 1, 2]])
('PQ', 'isConnected', 'SB'):tensor([[0],
        [0]])
('PQ', 'isConnected', 'PV'):tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  5,  9, 10, 11, 12,  5,  0],
        [ 0,  1,  2,  3,  4,  5,  6,  6,  8,  9, 10, 11,  5,  1, 14]])
('NB', 'isConnected', 'PV'):tensor([[0, 1],
        [0, 1]])
('NB', 'isConnected', 'PQ'):tensor([[0, 1, 2],
        [0, 1, 2]])
('PV', 'isConnected', 'PV'):tensor([[ 0,  1,  2,  3,  2,  5,  6,  5,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 17,  0, 23, 23,  5, 26,  6, 28,  5, 18, 31, 32,  5, 34, 23],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  0,  9, 10, 11, 12, 13,  9, 10, 16, 17,
         18, 19, 20, 18,

In [10]:
x = [[1,2,2,31],[1,2,25,31],[11,2,32,531],[1,22,2,311]]
#y = list(reversed(x))
t = torch.tensor(x)
t[torch.tensor([0,1])]

tensor([[ 1,  2,  2, 31],
        [ 1,  2, 25, 31]])

In [32]:
bus_idx_neighbors_dict["NB"]

{4: [('PV', 15, tensor([0.5852, 1.5820]))],
 0: [('PV', 14, tensor([1.0761, 2.9089])),
  ('PQ', 12, tensor([2.1526, 5.8188])),
  ('NB', 3, tensor([3.6198, 9.7850]))],
 2: [('PQ', 9, tensor([1.3074, 3.5341]))],
 1: [('PQ', 7, tensor([0.0587, 0.1587])), ('NB', 3, tensor([1.8857, 5.0973]))],
 3: [('NB', 1, tensor([1.8857, 5.0973])), ('NB', 0, tensor([3.6198, 9.7850]))]}

In [10]:
embedder_model(x_dict, constraint_dict, edge_idx_dict, edge_attr_dict)

defaultdict(torch.Tensor,
            {'PQ': tensor([[-0.9727,  0.4975,  0.1855,  0.1735],
                     [-0.9881, -0.3441,  0.1876,  0.1740],
                     [-0.9433, -0.1555,  0.5085,  0.3566],
                     [-0.8736, -0.6474,  0.8787,  0.5507],
                     [-0.9996,  2.1459,  0.1823,  0.1737],
                     [-0.7947, -0.3202,  0.1851,  0.1765],
                     [-1.0000,  6.4361,  0.7988,  0.5809],
                     [-0.8129, -0.4006,  0.1863,  0.1734],
                     [-0.9954,  4.7195,  0.1867,  0.1750],
                     [-1.0000,  3.5074,  0.1859,  0.1759],
                     [-0.9077, -0.3457,  0.5017,  0.3234],
                     [-0.9938,  0.8982,  0.4428,  0.3478],
                     [ 0.1649,  0.0599,  0.1850,  0.1719],
                     [-0.9517,  1.1979,  0.1877,  0.1745],
                     [-0.9716,  0.3346,  0.1855,  0.1764]], grad_fn=<CatBackward0>),
             'NB': tensor([[-0.5042, -0.9679,  0.1318,  0

In [6]:
np.sqrt(0.167**2 + 0.1685**2)

0.23723669614964715

In [7]:
for node_type in constraint_dict:
    print(custom_standard_inverse_transform(scaler, constraint_dict[node_type].detach().numpy()))

[[ 342.       418.       350.      -350.       350.      -350.
   350.00003]]
[[ 99.        121.          3.1069336   2.8781586   2.8781586   1.1700897
    1.1700897]
 [ 99.        121.          3.257019    3.0235138   3.0235138   1.2110443
    1.2110443]
 [ 99.        121.         37.16533     8.023071   34.423065    3.5772247
   14.011185 ]
 [ 99.        121.         51.020508   33.0876     46.6476     15.307114
   20.66623  ]
 [ 99.        121.          3.0006409   2.704483    2.704483    1.2998047
    1.2998047]
 [ 99.        121.          3.2549133   3.059372    3.059372    1.1112213
    1.1112213]
 [ 99.        121.         49.814194   31.622696   45.182693   15.616501
   20.975632 ]
 [ 99.        121.          3.423462    3.208374    3.208374    1.1942902
    1.1942902]
 [ 99.        121.          3.0209045   2.8138428   2.8138428   1.0991058
    1.0991058]
 [ 99.        121.          3.4769897   3.2351685   3.2351685   1.2740631
    1.2740631]
 [ 99.        121.         31.8847

In [ ]:
ACOPFOutput(x_dict, scalers_dict, None, index_mappers).output

In [ ]:
grid_names = sb.collect_all_simbench_codes()[25:]
acopf_ok_grid_names = []

for grid_name in grid_names:
    print(f"Trying grid {grid_name}")
    net = process_network(grid_name)

    # try:
    #     acopf_ok_grid_names.append(grid_name)
    #     pp.runpm_ac_opf(net)
    #     #print(net.res_bus)
    # except:
    #     print(f"Julia didnt converge for {grid_name}")
    #     acopf_ok_grid_names.remove(grid_name)
    try:
        acopf_ok_grid_names.append(grid_name)
        pp.runopp(net)
        #print(net.res_bus)
    except:
        print(f"OPP didnt converge for {grid_name}")
        acopf_ok_grid_names.remove(grid_name)
acopf_ok_grid_names
#save_multiple_unsupervised_inputs(grid_names, 100)

In [15]:
acopf_ok_grid_names = ['1-HV-mixed--1-sw','1-MV-semiurb--0-no_sw','1-MV-comm--0-sw','1-MV-comm--0-no_sw']
net = process_network(acopf_ok_grid_names[1])
pp.runopp(net)
net.res_bus

no costs are given - overall generated power is minimized


,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.025000,0.000000,-8.057173,-11.835759,1.000000,2.259515e-21
2,1.001425,209.119600,-1.176327,0.135443,1.000714,1.887024e-03
4,0.999942,209.185313,0.397800,0.183689,1.002495,4.950700e-03
5,0.999154,209.227646,0.293150,0.148678,1.003357,6.811706e-03
6,0.998564,209.265084,0.239906,0.171257,1.003935,8.382988e-03
...,...,...,...,...,...,...
112,0.987459,209.245271,0.237492,0.096853,1.020719,1.774877e-02
113,0.987333,209.251486,0.036984,0.085200,1.020904,1.802879e-02
114,0.987261,209.254787,-0.074431,0.034476,1.021013,1.818089e-02
115,0.987149,209.258063,0.118649,0.137547,1.021198,1.835478e-02


In [ ]:
acopf_grid_names = ['1-HV-mixed--0-no_sw','1-MV-semiurb--0-no_sw', '1-MV-comm--0-no_sw']
save_multiple_unsupervised_inputs(acopf_grid_names, num_samples=100)

In [4]:
inputs = load_multiple_unsupervised_inputs()
for i,input in enumerate(inputs):
    if input.res_bus is None:
        print(f"None at {i}")

In [ ]:
save_unsupervised_inputs('1-HV-mixed--0-no_sw', 300)

In [ ]:
inputs = load_unsupervised_inputs('1-HV-mixed--0-no_sw')
inputs[0].res_bus

In [ ]:
inputs

In [ ]:
inputs[57].res_bus

Display Plot of Customized Sigmoid Function

In [ ]:
import matplotlib.pyplot as p
import torch

def custom_tanh(x: torch.Tensor, lower_bound: float, upper_bound: float) -> torch.Tensor:
    width = upper_bound - lower_bound
    return 0.5 * width * torch.tanh(x) + 0.5 * (upper_bound + lower_bound)


lst = []
min_val = 0
max_val = 0.1
range_ = []
i = -2
while i < 2:
    range_.append(i)
    i += 0.01

for i in range_:
    val = custom_tanh(torch.tensor(i), min_val, max_val)
    #val = torch.tanh(torch.tensor(i))
    lst.append(val)

# Now, plot the values in lst
p.plot(range_, lst)
p.xlabel('Input')
p.ylabel('Value')
p.title('Plot of Enforcing Activation Function')
p.grid(True)
p.show()
print(range_)

Create Heterogeneous Self Supervised Model and Process Inputs

In [3]:
# Define the Parameters
grid_name = '1-HV-mixed--0-no_sw'


In [ ]:
# Load the Model
embedder_model = load_ACOPFGNN_model(grid_name, "embedder_model.pt", hidden_channels=4, num_layers=4)

In [ ]:
# Create ACOPFGNN model and Optimizer
index_mappers,net,data = generate_unsupervised_input(grid_name)
model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=1)

In [4]:
num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
learning_rate = 2.5e-4
optimizer = torch.optim.Adam(model.parameters())

In [5]:
num_parameters

2321

In [5]:
WANDB_NOTEBOOK_NAME = "msi"
# Initialize Weights & Biases
run = wandb.init(
        # set the wandb project where this run will be logged
        project="ACOPF-GNN-SelfSupervised-Hetero",

        # track hyperparameters and run metadata
        config={
            #"learning_rate": optimizer.,
            "architecture": "Hetero-SelfSupervised-GNN",
            "num_parameters": sum(p.numel() for p in model.parameters() if p.requires_grad),
            "num_heads": model.heads,
            "num_layers": model.num_layers,
            "grid_name": grid_name,
            "activation": model.act_fn,
            "dropout": model.dropout,
            "in_channels": model.in_channels,
            "output_channels": model.out_channels,
            "hidden_channels": model.hidden_channels,
            "channel type": "TransformerConv",
            "scaler": "MinMax",
            "model": "base_model_wired"
        }
    )


wandb: Currently logged in as: canbay96 (cbml). Use `wandb login --relogin` to force relogin


In [7]:
# Load Inputs
inputs = load_unsupervised_inputs(grid_name)
#inputs = load_multiple_unsupervised_inputs()
n = len(inputs)
train_inputs = inputs[:int(n*0.8)]
val_inputs = inputs[int(n*0.8): int(n*0.95)]
test_inputs = inputs[int(n*0.95):]



In [8]:
train_inputs[0].res_bus

{'SB': tensor([[ 6.0855, -1.1532,  6.4307, -4.8642]]),
 'PQ': tensor([[-0.2825,  1.3591,  0.1810,  0.1701],
         [-0.1253,  0.1567,  0.1793,  0.1708],
         [-0.1267,  0.1450,  0.3101,  0.2293],
         [-0.1348,  0.1020,  0.7808,  0.5394],
         [ 0.0020,  0.9540,  0.1793,  0.1688],
         [-0.2854,  1.0763,  0.1809,  0.1702],
         [-0.0670,  0.5362,  0.8061,  0.5207],
         [-0.1101,  1.3001,  0.1545,  0.1648],
         [-0.1840,  0.5053,  0.1789,  0.1704],
         [-0.1266,  0.1638,  0.1796,  0.1689],
         [-0.2876,  1.0932,  0.1815,  0.1698],
         [-0.1107,  0.5591,  0.3072,  0.2274],
         [-0.3032,  1.3241,  0.3103,  0.2452],
         [-0.2848,  1.0805,  0.1771,  0.1708],
         [-0.1255,  0.1539,  0.1796,  0.1717],
         [-0.1149,  1.3686,  0.1814,  0.1689]]),
 'PV': tensor([[ 4.5760e+00, -1.8228e-01,  5.3563e+00,  4.0652e+00],
         [ 1.5969e+00,  6.8250e-01,  3.4548e+00,  6.6139e+00],
         [-3.2272e-01,  1.2525e+00,  7.9409e-02,  1.1

In [7]:
# Train and Validate Model
start_epoch = 1
num_epochs = 100
loss_weights = (0.0, 1.0, 0.0)
model_name = "embedder_model_specialized.pt"
train_input,train_output,val_input, val_output = train_validate_ACOPF(model, model_name, optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

---------------   CURRENT LEARNING RATE: 0.001   ---------------
Epoch: 0
###########################################
   TRAINING
     Training Step: 0 Training Loss: 0.0 
     Training Step: 1 Training Loss: 0.0 
     Training Step: 2 Training Loss: 0.0 


KeyboardInterrupt: 

In [23]:
for i in range(61, 120, 20) :
    start_epoch = i
    num_epochs = 20 + i
    loss_weights = (1.0, 1.0, 1.0)
    model_name = "base_model_unsupervised_physics_constrained.pt"
    train_input,train_output,val_input, val_output = train_validate_ACOPF(model, model_name, optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

---------------   CURRENT LEARNING RATE: 4e-05   ---------------
Epoch: 60
###########################################
   TRAINING
     Training Step: 0 Training Loss: 6.849725723266602 
     Training Step: 1 Training Loss: 7.994863033294678 
     Training Step: 2 Training Loss: 7.418996334075928 
     Training Step: 3 Training Loss: 4.887266635894775 
     Training Step: 4 Training Loss: 5.012649059295654 
     Training Step: 5 Training Loss: 3.0778064727783203 
     Training Step: 6 Training Loss: 11.675814628601074 
     Training Step: 7 Training Loss: 10.785894393920898 
     Training Step: 8 Training Loss: 8.542866706848145 
     Training Step: 9 Training Loss: 9.035882949829102 
     Training Step: 10 Training Loss: 9.545356750488281 
     Training Step: 11 Training Loss: 8.43463134765625 
     Training Step: 12 Training Loss: 12.55498218536377 
     Training Step: 13 Training Loss: 5.320827484130859 
     Training Step: 14 Training Loss: 6.62136173248291 
     Training Step: 15 

In [8]:
print(sum(train_output.output[:, 2]), sum(train_output.output[:, 3]))
train_output.output

-593.8486800193787 -243.53982478380203


array([[ 9.08173089e-01, -1.83501959e+00, -5.90927734e+01,
        -4.32426147e+01],
       [ 9.80060136e-01,  0.00000000e+00,  9.09702148e+01,
        -3.20258598e+01],
       [ 1.03807630e+00, -1.48351908e+00, -5.60315399e+01,
        -4.20662727e+01],
       [ 1.03758441e+00,  1.54328156e+00, -8.17986012e-01,
         7.37901390e-01],
       [ 1.02849822e+00,  1.60070038e+00,  2.79828215e+00,
         8.60434711e-01],
       [ 1.02379331e+00,  1.61828518e+00,  2.95484495e+00,
         1.07658315e+00],
       [ 1.05319075e+00,  1.20048451e+00, -6.75554800e+00,
        -1.33368182e+00],
       [ 1.04970037e+00,  9.24345732e-01, -1.38833284e+01,
        -2.18041611e+00],
       [ 1.05298337e+00,  1.13721991e+00, -8.29765701e+00,
        -1.68316507e+00],
       [ 1.03909489e+00,  2.39133453e+00,  2.30472603e+01,
         6.45330811e+00],
       [ 1.06325094e+00, -8.12291145e-01, -5.12167511e+01,
        -1.27561131e+01],
       [ 1.06275891e+00,  5.54805279e-01, -2.24758530e+01,
      

In [9]:
print(sum(val_output.output[:, 2]), sum(val_output.output[:, 3]))
val_output.output

-511.95279839634895 -215.83535793423653


array([[ 9.07937301e-01, -1.87421203e+00, -5.86987801e+01,
        -4.31370201e+01],
       [ 9.78537469e-01,  0.00000000e+00,  9.15387573e+01,
        -3.18711929e+01],
       [ 1.03759433e+00, -1.51700187e+00, -5.56474800e+01,
        -4.19558372e+01],
       [ 1.03786753e+00,  1.56929421e+00, -2.91552931e-01,
         5.73150635e-01],
       [ 1.02684687e+00,  1.61438823e+00,  2.79138613e+00,
         9.32665348e-01],
       [ 1.02212753e+00,  1.63119197e+00,  2.91438818e+00,
         1.13415098e+00],
       [ 1.05419103e+00,  1.20319247e+00, -6.55964136e+00,
        -1.39674544e+00],
       [ 1.05039056e+00,  9.21392202e-01, -1.37815561e+01,
        -2.24824166e+00],
       [ 1.05342338e+00,  1.15047312e+00, -7.85218763e+00,
        -1.60835695e+00],
       [ 1.03538874e+00,  2.41840339e+00,  2.30585728e+01,
         6.94025660e+00],
       [ 1.06390915e+00, -8.37588310e-01, -5.08798523e+01,
        -1.26762056e+01],
       [ 1.06576177e+00,  5.09796381e-01, -2.31083965e+01,
      

In [ ]:
def save_ACOPFGNN_model(model: ACOPFGNN):
    path = r"./Models/SelfSupervised/hetero_model_unsupervised.pt"

    torch.save(model.state_dict(), path)
save_ACOPFGNN_model(embedder_model)

In [ ]:
train_output.output

CHAINED ACOPF MODELS

-

-

-

-

In [2]:
# Define the Parameters
grid_name = '1-HV-mixed--0-no_sw'
embedder_model = load_ACOPFGeneral_model(grid_name, "hetero_model_bus.pt",16,1)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'SB': tensor([[-0.9018,  0.6162, -0.5444,  0.6173]], grad_fn=<ViewBackward0>), 'PQ': tensor([[ 2.3276, -0.0358,  1.1119,  1.2931],
        [ 1.4120,  0.0319,  5.0174, -1.9771],
        [-4.0095,  4.6728,  3.7275, -0.0968],
        [ 4.4933,  0.9351,  1.4761,  1.5370],
        [-2.6381,  0.3272,  0.2074, -1.3705],
        [ 4.6313,  4.3858,  4.6025,  1.8075],
        [-0.8091,  0.2785, -1.5325, -7.5600],
        [ 3.9265, -1.6268, -0.5522, -2.7840],
        [ 3.3903, -3.8023,  3.9481,  3.2486],
        [ 2.7211,  0.0281,  2.6178, -0.4197],
        [-1.0556, -2.2014, -1.2151, -0.9475],
        [ 1.0045, -2.1738, -0.3324,  0.2518],
        [-6.1500, -5.2787,  4.1116, -2.1850],
        [ 3.4343,  0.1623, -3.5839, -0.8874],
        [-4.1475, -0.2775,  0.0306,  1.4033],
        [ 3.8743, -0.4185, -4.1092,  1.2905]], grad_fn=<ViewBackward0>), 'PV': tensor([[ 1.3684, -0.7577, -2.1886, -0.1629],
        [-0.4488, -0.1512, -2.2093,  1.1575],
        [ 0.0744,  0.9206

In [2]:
# Create Minimizer, Enforcer and Embedder Models
# Define the Parameters
grid_name = '1-HV-mixed--0-no_sw'
index_mappers,net,data = generate_unsupervised_input(grid_name)
#minimizer_model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=1)
#embedder_model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=1)

#minimizer_model = create_ACOPFGNN_model(data,
# net, index_mappers, hidden_channels=4, num_layers=4)

#embedder_model = create_ACOPFEmbedder_model(data, net, index_mappers, hidden_channels=4, num_layers=1)

embedder_model = create_ACOPFEmbedder_Bus_Constrained(data, net, index_mappers, hidden_channels=16, num_layers=1)#create_ACOPFGeneral_model(data, net, index_mappers, hidden_channels=16, num_layers=1)


#enforcer_model = create_ACOPFEnforcer_model(data, net, index_mappers, hidden_channels=4, num_layers=4)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'SB': tensor([[ 5.9153, -0.1487, -1.9773,  6.3468]], grad_fn=<CatBackward0>), 'PQ': tensor([[-0.2141, -0.0308,  0.2188,  0.1954],
        [-0.0782, -0.5282,  0.2190,  0.1954],
        [-0.4411,  0.3749,  0.4222,  0.4817],
        [-0.0600,  1.5440,  0.8493,  0.5651],
        [-0.2426, -1.8049,  0.2189,  0.1970],
        [-0.0842, -0.4986,  0.2209,  0.1980],
        [-0.3681, -0.0104,  0.8526,  0.7059],
        [-0.4406,  0.8670,  0.9460,  0.2177],
        [-0.1032, -1.5601,  0.2229,  0.1970],
        [-0.2986, -1.9646,  0.2227,  0.1973],
        [-0.2111,  1.1219,  0.2204,  0.1967],
        [-0.4429,  1.3490,  0.3457,  0.4921],
        [-0.3948,  0.0455,  0.7214,  0.3177],
        [-0.2853,  0.2446,  0.2199,  0.1980],
        [-0.4306, -1.1787,  0.2182,  0.1972],
        [-0.3970, -0.5784,  0.2182,  0.1964]], grad_fn=<CatBackward0>), 'PV': tensor([[ 4.8654e+00,  3.5560e+00, -4.3368e+00,  1.5640e-01],
        [ 2.4065e+00, -3.4883e+00,  1.6011e-01,  1.6452e-

In [7]:
# Initialize the Optimizer
from itertools import chain
#optimizer = torch.optim.Adam(chain(minimizer_model.parameters(), enforcer_model.parameters(), embedder_model.parameters()))
ACOPF_optimizer = torch.optim.Adam(embedder_model.parameters(), lr=1e-4)


In [54]:
ACOPF_optimizer.param_groups[0]['lr']*=0.5

In [4]:
WANDB_NOTEBOOK_NAME = "msi"
# Initialize Weights & Biases
run = wandb.init(
        # set the wandb project where this run will be logged
        project="ACOPF-GNN-SelfSupervised-Hetero",

        # track hyperparameters and run metadata
        config={
            #"learning_rate": optimizer.,
            "architecture": "Hetero-SelfSupervised-GNN",
            "grid_name": grid_name,
            "Training Type": "Supervised",
            "model": "hetero-bus-constrained"

        }
    )


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: canbay96 (cbml). Use `wandb login --relogin` to force relogin


In [5]:
# Load Inputs
inputs = load_unsupervised_inputs(grid_name)
#inputs = load_multiple_unsupervised_inputs()
n = len(inputs)
train_inputs = inputs[:int(n*0.8)]
val_inputs = inputs[int(n*0.8): int(n*0.95)]
test_inputs = inputs[int(n*0.95):]

In [6]:
# Train and Validate Model
start_epoch = 1
num_epochs = 75
loss_weights = (1.0, 1.0, 1.0)
train_input,train_output,val_input, val_output = train_validate_ACOPF_chained(None,None, embedder_model,ACOPF_optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

Epoch: 0
###########################################
   TRAINING
     Training Step: 0 Training Loss: 1.611656904220581 
     Training Step: 1 Training Loss: 1.4702457189559937 
     Training Step: 2 Training Loss: 1.35362708568573 
     Training Step: 3 Training Loss: 1.2682572603225708 
     Training Step: 4 Training Loss: 1.20698881149292 
     Training Step: 5 Training Loss: 1.1645708084106445 
     Training Step: 6 Training Loss: 1.1335264444351196 
     Training Step: 7 Training Loss: 1.1162726879119873 
     Training Step: 8 Training Loss: 1.0983985662460327 
     Training Step: 9 Training Loss: 1.080711007118225 
     Training Step: 10 Training Loss: 1.0645232200622559 
     Training Step: 11 Training Loss: 1.0498509407043457 
     Training Step: 12 Training Loss: 1.0334110260009766 
     Training Step: 13 Training Loss: 1.01816987991333 
     Training Step: 14 Training Loss: 1.005792260169983 
     Training Step: 15 Training Loss: 0.9963366389274597 
     Training Step: 16 Tra

In [8]:
# Train and Validate Model
start_epoch = 76
num_epochs = 150
loss_weights = (1.0 ,1.0, 1.0)
train_input,train_output,val_input, val_output = train_validate_ACOPF_chained(None,None, embedder_model,ACOPF_optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

Epoch: 75
###########################################
   TRAINING
     Training Step: 0 Training Loss: 0.6210901737213135 
     Training Step: 1 Training Loss: 0.6127399206161499 
     Training Step: 2 Training Loss: 0.6145259141921997 
     Training Step: 3 Training Loss: 0.619731068611145 
     Training Step: 4 Training Loss: 0.6151012182235718 
     Training Step: 5 Training Loss: 0.6166698932647705 
     Training Step: 6 Training Loss: 0.6152848601341248 
     Training Step: 7 Training Loss: 0.6185854077339172 
     Training Step: 8 Training Loss: 0.6166726350784302 
     Training Step: 9 Training Loss: 0.6107650399208069 
     Training Step: 10 Training Loss: 0.6117033362388611 
     Training Step: 11 Training Loss: 0.6114595532417297 
     Training Step: 12 Training Loss: 0.6180527210235596 
     Training Step: 13 Training Loss: 0.6164128184318542 
     Training Step: 14 Training Loss: 0.6140622496604919 
     Training Step: 15 Training Loss: 0.6106172800064087 
     Training Ste

In [21]:
train_output.res_bus

{'SB': tensor([[ 6.0855, -1.1536,  6.4379, -4.8384]]),
 'PQ': tensor([[-0.2824,  1.3603,  0.1809,  0.1686],
         [-0.1250,  0.1585,  0.1808,  0.1706],
         [-0.1259,  0.1522,  0.2675,  0.2229],
         [-0.1347,  0.1003,  0.8072,  0.5386],
         [ 0.0017,  0.9427,  0.1777,  0.1698],
         [-0.2852,  1.0777,  0.1790,  0.1704],
         [-0.0672,  0.5336,  0.7829,  0.5605],
         [-0.1090,  1.2983,  0.1582,  0.1465],
         [-0.1837,  0.5072,  0.1779,  0.1705],
         [-0.1263,  0.1656,  0.1798,  0.1688],
         [-0.2874,  1.0945,  0.1818,  0.1696],
         [-0.1101,  0.5598,  0.3081,  0.2222],
         [-0.3022,  1.3244,  0.3070,  0.2200],
         [-0.2846,  1.0818,  0.1790,  0.1714],
         [-0.1251,  0.1578,  0.1781,  0.1715],
         [-0.1140,  1.3667,  0.1820,  0.1702]]),
 'PV': tensor([[ 4.5760e+00, -1.8209e-01,  5.3566e+00,  4.0813e+00],
         [ 1.5967e+00,  6.8131e-01,  3.4700e+00,  6.6232e+00],
         [-3.2277e-01,  1.2543e+00,  8.1089e-02,  1.1